In [2]:
import requests
import pandas as pd
import geopandas as gpd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
from shapely import wkt
import folium
from folium import LayerControl, GeoJson, GeoJsonTooltip
import branca

# Fetch data from the Flask API
def fetch_data():
    try:
        response = requests.get('http://127.0.0.1:5000/map')
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Landslide Risk Dashboard', style={'text-align': 'center'}),

    # Dropdowns for risk type and level
    html.Div([
        html.Label('Risk Type:', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='risk-type-dropdown',
            options=[
                {'label': 'Surface Area', 'value': 'surface_area'},
                {'label': 'Population', 'value': 'population'},
                {'label': 'Buildings', 'value': 'buildings'}
            ],
            value='surface_area'
        ),
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label('Risk Level:', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='risk-level-dropdown',
            options=[
                {'label': 'High and Very High', 'value': 'high_very_high'},
                {'label': 'Moderate', 'value': 'moderate'},
                {'label': 'Low', 'value': 'low'}
            ],
            value='high_very_high'
        ),
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),

    # Opacity slider
    html.Div([
        html.Label('Opacity:', style={'font-weight': 'bold'}),
        dcc.Slider(
            id='opacity-slider',
            min=0,
            max=1,
            step=0.01,
            value=0.7,
            marks={0: '0', 1: '1'},
            tooltip={'always_visible': True, 'placement': 'bottom'}
        ),
        html.Div(id='opacity-output-container', style={'margin-top': '10px'})
    ], style={'width': '30%', 'padding': '20px', 'margin': 'auto', 'text-align': 'center'}),

    # Loading indicator
    dcc.Loading(
        id="loading",
        type="default",
        children=[
            html.Div(id='map-container', children=[
                html.Iframe(id='map', srcDoc=None, width='100%', height='600')
            ])
        ]
    )
])

# Callback to update the map and opacity
@app.callback(
    Output('map', 'srcDoc'),
    [Input('risk-type-dropdown', 'value'),
     Input('risk-level-dropdown', 'value'),
     Input('opacity-slider', 'value')]
)
def update_map(risk_type, risk_level, opacity):
    df = fetch_data()
    
    if df.empty:
        return None  # Return None if data fetching failed

    # Convert WKT to GeoDataFrame
    df['geometry'] = df['geom_wkt'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Set the initial CRS to EPSG:32632 and then transform to EPSG:4326
    gdf.set_crs('EPSG:32632', inplace=True)
    gdf = gdf.to_crs(epsg=4326)

    # Drop the 'geom_wkt' column
    gdf = gdf.drop(columns=['geom_wkt'])

    # Map risk type and level to the corresponding column
    risk_columns = {
        'surface_area': {
            'high_very_high': 'ar_fr_p3p4',
            'moderate': 'ar_fr_p2',
            'low': 'ar_fr_p1'
        },
        'population': {
            'high_very_high': 'popfrp3p4p',
            'moderate': 'pop_fr_p2',
            'low': 'pop_fr_p1'
        },
        'buildings': {
            'high_very_high': 'ed_fr_p3p4',
            'moderate': 'ed_fr_p2',
            'low': 'ed_fr_p1'
        }
    }

    selected_column = risk_columns[risk_type][risk_level]
    print("Selected column:", selected_column)  # Print the selected column for debugging

    # Define custom colormaps for each risk type using ColorBrewer palettes
    colormaps = {
        'surface_area': branca.colormap.LinearColormap(
            colors=['#f7fbff', '#c6dbef', '#6baed6', '#3182bd', '#08519c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Surface Area Risk Level"
        ),
        'population': branca.colormap.LinearColormap(
            colors=['#f7fcf5', '#c7e9c0', '#74c476', '#31a354', '#006d2c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Population Risk Level"
        ),
        'buildings': branca.colormap.LinearColormap(
            colors=['#fff5f0', '#fcbba1', '#fc9272', '#fb6a4a', '#cb181d'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Buildings Risk Level"
        )
    }

    colormap = colormaps[risk_type]

    # Create a map
    m = folium.Map(location=[45.74548533557156, 9.680328369140627], zoom_start=7)

    # Add base tile layers directly
    folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr='Map data &copy; OpenStreetMap contributors',
        name='OpenStreetMap',
        overlay=False
    ).add_to(m)

    folium.TileLayer(
        tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
        attr='Map data: &copy; OpenTopoMap contributors',
        name='Esri Topo',
        overlay=False
    ).add_to(m)

    folium.TileLayer(
        tiles='https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png',
        attr='Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        name='CartoDB Voyager',
        overlay=False
    ).add_to(m)

    folium.TileLayer(
        tiles='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
        attr='Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        name='CartoDB Dark',
        overlay=False
    ).add_to(m)

    # Add a choropleth layer to the map
    def style_function(feature):
        return {
            'fillColor': colormap(feature['properties'][selected_column]),
            'color': 'black',
            'fillOpacity': opacity,
            'weight': 0.5
        }

    folium.GeoJson(
        gdf,
        name='Choropleth',
        style_function=style_function,
        tooltip=GeoJsonTooltip(
            fields=['nome', selected_column],
            aliases=['Region', 'Value'],
            localize=True,
            sticky=True,
            opacity=0.8,
            direction='top'
        )
    ).add_to(m)

    # Add layer control and colormap to the map
    LayerControl().add_to(m)
    colormap.add_to(m)

    return m.get_root().render()

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8081)


Selected column: ar_fr_p3p4
Selected column: ar_fr_p3p4
Selected column: ar_fr_p3p4
Selected column: popfrp3p4p
Selected column: ed_fr_p3p4
Selected column: ed_fr_p3p4
Selected column: ed_fr_p3p4
Selected column: popfrp3p4p
Selected column: popfrp3p4p
Selected column: popfrp3p4p
Selected column: popfrp3p4p
